In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# move into project directory
repo_name = "UMass-CS670-Project"
%cd /content/drive/MyDrive/UMass-Courses/Semester-2/CS670/$repo_name
!ls

/content/drive/MyDrive/UMass-Courses/Semester-2/CS670/UMass-CS670-Project
common			  experiments		   index.py		  README.md
datautils		  get_dataset.sh	   last_run_info.json	  SINet.ipynb
demo			  github-repo-clone.ipynb  models		  source-data
excluded-train-files.txt  Index.ipynb		   preprocess_dataset.py  texture_params.txt


In [ ]:
# set up environment
# comment out if not required
'''
!pip3 install torch torchvision torchaudio
!pip install matplotlib numpy pandas pyyaml opencv-python
'''

'\n!pip3 install torch torchvision torchaudio\n!pip install matplotlib numpy pandas pyyaml opencv-python\n'

In [ ]:
# this cell is for downloading data.
# as of yet data is not hosted and is available in the private data folder
# uncomment the lines below  to download the data

import os

!pip install gdown

data_dir="source-data"

if not(os.path.exists("source-data")):
    !gdown https://drive.google.com/uc?id=18oBjWeuw5qAq4HG_jZdjUHas4APy-KJE&export=download
    #echo 'Downloaded data! Unzipping to data folder'
    !unzip -qq -d . ./COD10K-v3.zip
    os.rename("./COD10K-v3.zip", "./source-data/COD10K-v3.zip")
else:
    print("\nThe data directory exists!")

Downloading...
From (original): https://drive.google.com/uc?id=18oBjWeuw5qAq4HG_jZdjUHas4APy-KJE
From (redirected): https://drive.google.com/uc?id=18oBjWeuw5qAq4HG_jZdjUHas4APy-KJE&confirm=t&uuid=08ee156c-0356-48f7-84f5-ca61d0cc5469
To: /content/drive/MyDrive/UMass-Courses/Semester-2/CS670/UMass-CS670-Project/COD10K-v3.zip
100% 2.38G/2.38G [00:38<00:00, 62.4MB/s]


In [ ]:
# setup some imports
from datautils.datareader import read_data
from datautils.dataset import COD10KDataset
from torch.utils.data import DataLoader
import random
import numpy as np
import torch
import argparse
from tqdm import tqdm
from torchvision.io import read_image
from torchvision.utils import save_image
from torchvision.transforms.functional import to_pil_image
import json

from experiments.style_transfer import style_transfer
from common.visualizer import layer_visualizer

In [ ]:
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
def run_style_transfer_pipeline(args, texture_name, style_weight, last_batch_run = -1):
    pos_data_paths = read_data('Train')

    dataset = COD10KDataset(pos_data_paths)
    dataloader = DataLoader(dataset, batch_size = args.batch_size, shuffle = False)

    for i_batch, batch in enumerate(dataloader):
        if i_batch > last_batch_run:
            print(f"Processing batch {i_batch}, image: {batch['img_name']} of dimensions: {batch['img'].shape}")
            style_img = read_image(f'./source-data/Textures/{texture_name}.jpg')
            new_img = style_transfer(
                batch['img'],
                style_img,
                [0, 2, 5, 14, 23],
                21,
                1e-4,
                [style_weight]*5,
                1e-5,
                args)
            img_name = batch['img_name'][0]
            img_name = img_name.replace(".jpg", "")
            img_name = f"./source-data/Train/Styled-Image/{img_name}-Texture-{texture_name}.jpg"
            save_image(new_img, img_name)
            last_run = {
                "last_batch": i_batch
            }
            print(f"completing transfer of img {i_batch} with texture {texture_name}")
            with open("./last_run_info.json", "w") as fp:
                json.dump(last_run, fp)

    '''
    style_img = read_image(f'./data/Textures/{texture_name}.jpg')
    content_img = read_image('./data/Mini-Set/butterfly-image.jpg')
    #content_img = batch['img']
    print('b4 trans', content_img.shape)
    new_img = style_transfer(content_img,
            style_img,
            [0, 2, 5, 14, 23],
            21,
            1e-4,
            [style_weight]*5,
            1e-5,
            args)
    #pi = to_pil_image(pi)
    #plt.axis("off")
    #plt.imshow(pi)
    #plt.show()
    save_image(new_img, f'./data/COD10K-v3/Train/Styled-Image/butterfly-image-Texture-{texture_name}.jpg')
    #print('after trans', new_img.size())
    '''

In [ ]:

param_dict = {
    'moss': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'snow': {
        'epochs': 200,
        'style_weights': 1.5
    },
    'wet-sand': {
        'epochs': 200,
        'style_weights': 3.5
    },
    'sand-with-moss': {
        'epochs': 100,
        'style_weights': 2.5
    },
    'rain': {
        'epochs': 100,
        'style_weights': 4.5
    },
    'wood': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'grass': {
        'epochs': 100,
        'style_weights': 0.1
    },
    'foliage-texture': {
        'epochs': 100,
        'style_weights': 0.5
    },
    'blue-coral': {
        'epochs': 100,
        'style_weights': 1.1
    }
}

with open("./last_run_info.json") as fp:
    last_run = json.load(fp)["last_batch"]

selected_textures = ['moss', 'wet-sand', 'sand-with-moss', 'rain', 'foliage-texture']

for texture in selected_textures:
    parser = argparse.ArgumentParser()
    parser.add_argument('--batch_size', type = int, default = 1)
    parser.add_argument('--model_name', type=str, default='vgg')
    parser.add_argument('--lr', type=float, default=0.05)
    parser.add_argument('--max_iter', type=int, default=param_dict[texture]['epochs'])
    args = parser.parse_args(args=[])

    #style_img = read_image(f'./data/Textures/{texture}.jpg')
    #print(img.size())
    #layer_visualizer(img, args)

    run_style_transfer_pipeline(args, texture, param_dict[texture]['style_weights'], last_run)

Processing batch 710, image: ['COD10K-CAM-3-Flying-54-Bittern-3279.jpg'] of dimensions: torch.Size([1, 3, 765, 1024])


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:03<00:00, 154MB/s]


completing transfer of img 710 with texture moss
Processing batch 711, image: ['COD10K-CAM-2-Terrestrial-23-Cat-1468.jpg'] of dimensions: torch.Size([1, 3, 552, 770])
completing transfer of img 711 with texture moss
Processing batch 712, image: ['COD10K-CAM-3-Flying-61-Katydid-4028.jpg'] of dimensions: torch.Size([1, 3, 982, 1005])
completing transfer of img 712 with texture moss
Processing batch 713, image: ['COD10K-CAM-1-Aquatic-3-Crab-49.jpg'] of dimensions: torch.Size([1, 3, 576, 768])
completing transfer of img 713 with texture moss
Processing batch 714, image: ['COD10K-CAM-1-Aquatic-9-GhostPipefish-319.jpg'] of dimensions: torch.Size([1, 3, 581, 1016])
completing transfer of img 714 with texture moss
Processing batch 715, image: ['COD10K-CAM-1-Aquatic-14-ScorpionFish-920.jpg'] of dimensions: torch.Size([1, 3, 600, 800])
completing transfer of img 715 with texture moss
Processing batch 716, image: ['COD10K-CAM-3-Flying-55-Butterfly-3432.jpg'] of dimensions: torch.Size([1, 3, 591, 

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.17 GiB. GPU 0 has a total capacity of 14.75 GiB of which 665.06 MiB is free. Process 2743 has 14.10 GiB memory in use. Of the allocated memory 12.30 GiB is allocated by PyTorch, and 1.65 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)